In [6]:
import pandas as pd
import json
from pandas.io.json import json_normalize

In [7]:
import MySQLdb as db
from pandas import DataFrame
#from pandas.io.sql import frame_query

database = db.connect('69.89.14.187','engage','punya123','engage_v1')
#data     = frame_query("SELECT * FROM ass_v1_360_invite_response", database)

In [8]:
database.autocommit=False

In [9]:
sql_all_mgr = """select
            surveyrepo9_.INDEX_TYPE as col_0_0_,
            avg(surveyopti2_.OPT_VALUE) as col_1_0_ 
        from
            survey_v1_user_response_store surveyuser0_ 
        inner join
            survey_v1_user_servey_def surveyuser1_ 
                on surveyuser0_.USER_SURVEY_DEF_ID=surveyuser1_.USD_ID 
        inner join
            survey_v1_defination surveydefi6_ 
                on surveyuser1_.SURVEY_DEF_ID=surveydefi6_.SD_ID 
        inner join
            manager_survey_user_def_mapping managersur7_ 
                on surveyuser1_.USD_ID=managersur7_.SUSD_ID 
        inner join
            survey_manager surveymana8_ 
                on managersur7_.MANAGER_ID=surveymana8_.MANAGER_ID 
        inner join
            survey_v1_option surveyopti2_ 
                on surveyuser0_.OPTION_ID=surveyopti2_.OPT_ID 
        inner join
            survey_v1_question surveyques3_ 
                on surveyuser0_.QUESTION_ID=surveyques3_.QUESTION_ID 
        inner join
            survey_v1_report_index_question_mapper surveyrepo4_ 
                on surveyques3_.QUESTION_ID=surveyrepo4_.QUESTION_ID 
        inner join
            survey_v1_report_index surveyrepo5_ 
                on surveyrepo4_.REPORT_INDEX_ID=surveyrepo5_.REPORT_INDEX_ID,
            survey_v1_report_index surveyrepo9_ 
        where
            surveyrepo4_.REPORT_INDEX_ID=surveyrepo9_.REPORT_INDEX_ID 
            and (
                surveyopti2_.OPT_ID is not null
            ) 
            and surveyques3_.QUESTION_TYPE='QUESTION_SCALE' 
            and surveydefi6_.SD_ID=12 
            and surveymana8_.MANAGER_EMP_ID=7746 
        group by
            surveyrepo5_.INDEX_TYPE"""

In [10]:
mgr_all = pd.read_sql_query(sql=sql_all_mgr,con=database)

In [11]:
mgr_all

,col_0_0_,col_1_0_
0,C,4.201871
1,I,3.986634


In [12]:
sql_old = """SELECT surveymana8_.MANAGER_EMP_ID,
       surveyrepo9_.INDEX_TYPE AS col_0_0_,
       avg(surveyopti2_.OPT_VALUE) AS col_1_0_
  FROM survey_v1_user_response_store surveyuser0_
       INNER JOIN survey_v1_user_servey_def surveyuser1_
          ON surveyuser0_.USER_SURVEY_DEF_ID = surveyuser1_.USD_ID
       INNER JOIN survey_v1_defination surveydefi6_
          ON surveyuser1_.SURVEY_DEF_ID = surveydefi6_.SD_ID
       INNER JOIN manager_survey_user_def_mapping managersur7_
          ON surveyuser1_.USD_ID = managersur7_.SUSD_ID
       INNER JOIN survey_manager surveymana8_
          ON managersur7_.MANAGER_ID = surveymana8_.MANAGER_ID
       INNER JOIN survey_v1_option surveyopti2_
          ON surveyuser0_.OPTION_ID = surveyopti2_.OPT_ID
       INNER JOIN survey_v1_question surveyques3_
          ON surveyuser0_.QUESTION_ID = surveyques3_.QUESTION_ID
       INNER JOIN survey_v1_report_index_question_mapper surveyrepo4_
          ON surveyques3_.QUESTION_ID = surveyrepo4_.QUESTION_ID
       INNER JOIN survey_v1_report_index surveyrepo5_
          ON surveyrepo4_.REPORT_INDEX_ID = surveyrepo5_.REPORT_INDEX_ID,
       survey_v1_report_index surveyrepo9_
 WHERE     surveyrepo4_.REPORT_INDEX_ID = surveyrepo9_.REPORT_INDEX_ID
       AND (surveyopti2_.OPT_ID IS NOT NULL)
       AND surveyques3_.QUESTION_TYPE = 'QUESTION_SCALE'
       AND surveydefi6_.SD_ID = 12      
GROUP BY surveymana8_.MANAGER_EMP_ID,surveyrepo5_.INDEX_TYPE"""
old_data = pd.read_sql_query(sql=sql_old,con=database)
old_data.head()
old_data = old_data.rename(columns={"col_0_0_":"INDEX_TYPE_OLD","col_1_0_":"VALUE_OLD"})

In [13]:
old_data.head()

,MANAGER_EMP_ID,INDEX_TYPE_OLD,VALUE_OLD
0,1,C,4.118326
1,1,I,3.976470
2,10028,C,4.519651
3,10028,I,4.316206
4,10043,C,3.550000


In [14]:
sql_new = """SELECT user_info.EMP_ID,surveyrepo9_.INDEX_TYPE AS col_0_0_,
       avg(surveyopti2_.OPT_VALUE) AS col_1_0_
  FROM survey_v1_user_response_store surveyuser0_
       INNER JOIN survey_v1_user_servey_def surveyuser1_
          ON surveyuser0_.USER_SURVEY_DEF_ID = surveyuser1_.USD_ID
       JOIN all_survey_v1_user_client_mapper_view
          ON all_survey_v1_user_client_mapper_view.UCM_ID =
                surveyuser1_.UCM_ID
       JOIN user_info
          ON user_info.USER_ID =
                all_survey_v1_user_client_mapper_view.USER_ID
       INNER JOIN survey_v1_defination surveydefi6_
          ON surveyuser1_.SURVEY_DEF_ID = surveydefi6_.SD_ID
       INNER JOIN survey_v1_option surveyopti2_
          ON surveyuser0_.OPTION_ID = surveyopti2_.OPT_ID
       INNER JOIN survey_v1_question surveyques3_
          ON surveyuser0_.QUESTION_ID = surveyques3_.QUESTION_ID
       INNER JOIN survey_v1_report_index_question_mapper surveyrepo4_
          ON surveyques3_.QUESTION_ID = surveyrepo4_.QUESTION_ID
       INNER JOIN survey_v1_report_index surveyrepo5_
          ON surveyrepo4_.REPORT_INDEX_ID = surveyrepo5_.REPORT_INDEX_ID,
       survey_v1_report_index surveyrepo9_
 WHERE     surveyrepo4_.REPORT_INDEX_ID = surveyrepo9_.REPORT_INDEX_ID
       AND (surveyopti2_.OPT_ID IS NOT NULL)
       AND surveyques3_.QUESTION_TYPE = 'QUESTION_SCALE'
       AND surveydefi6_.SD_ID = 12    
GROUP BY user_info.EMP_ID,surveyrepo5_.INDEX_TYPE"""
new_data = pd.read_sql_query(sql=sql_new,con=database)
new_data.head()
new_data = new_data.rename(columns={"col_0_0_":"INDEX_TYPE_NEW","col_1_0_":"VALUE_NEW"})
new_data.head()

,EMP_ID,INDEX_TYPE_NEW,VALUE_NEW
0,10027,C,4.421053
1,10027,I,4.272727
2,10028,C,4.400000
3,10028,I,4.318182
4,10030,C,4.900000


In [15]:
new_data.sort_values("EMP_ID").head()

,EMP_ID,INDEX_TYPE_NEW,VALUE_NEW
0,10027,C,4.421053
1,10027,I,4.272727
2,10028,C,4.400000
3,10028,I,4.318182
4,10030,C,4.900000


In [16]:
join_data = pd.merge(old_data,new_data,left_on=["MANAGER_EMP_ID","INDEX_TYPE_OLD"],right_on=["EMP_ID","INDEX_TYPE_NEW"])

In [17]:
join_data  = join_data.sort_values("MANAGER_EMP_ID")

In [18]:
join_data["DIFF"]=join_data.VALUE_NEW-join_data.VALUE_OLD

In [19]:
join_data.head()

,MANAGER_EMP_ID,INDEX_TYPE_OLD,VALUE_OLD,EMP_ID,INDEX_TYPE_NEW,VALUE_NEW,DIFF
0,10028,C,4.519651,10028,C,4.400000,-0.119651
1,10028,I,4.316206,10028,I,4.318182,0.001976
2,10043,C,3.550000,10043,C,4.450000,0.900000
3,10043,I,3.295455,10043,I,4.136364,0.840909
4,10060,C,4.246493,10060,C,4.250000,0.003507


In [20]:
join_data.drop(['EMP_ID','INDEX_TYPE_OLD'],axis=1,inplace=True)
join_data.head()

,MANAGER_EMP_ID,VALUE_OLD,INDEX_TYPE_NEW,VALUE_NEW,DIFF
0,10028,4.519651,C,4.400000,-0.119651
1,10028,4.316206,I,4.318182,0.001976
2,10043,3.550000,C,4.450000,0.900000
3,10043,3.295455,I,4.136364,0.840909
4,10060,4.246493,C,4.250000,0.003507


In [21]:
join_data[join_data.MANAGER_EMP_ID.isin(['2772','20530','2967','26580'])]

,MANAGER_EMP_ID,VALUE_OLD,INDEX_TYPE_NEW,VALUE_NEW,DIFF
399,20530,3.543933,C,4.800000,1.256067
400,20530,3.604563,I,4.636364,1.031801
657,2772,3.782828,C,0.000000,-3.782828
658,2772,3.802752,I,0.000000,-3.802752
670,2967,4.267409,C,0.000000,-4.267409
671,2967,4.197468,I,0.000000,-4.197468


In [22]:
#join_data.to_excel("F:\GMR_MANAGER_COMPARE_REPORT.xlsx",sheet_name="comparision")

In [23]:
sql_emp = """SELECT survey_manager.MANAGER_EMP_ID,
       survey_v1_report_index.INDEX_TYPE AS 'Survey_Type',
       AVG(`survey_v1_option`.OPT_VALUE) AS 'Avg_Score'
  FROM engage_v1.survey_v1_user_response_store
       JOIN survey_v1_user_servey_def
          ON survey_v1_user_servey_def.USD_ID =
                survey_v1_user_response_store.USER_SURVEY_DEF_ID
       JOIN survey_v1_question
          ON survey_v1_question.QUESTION_ID =
                survey_v1_user_response_store.QUESTION_ID
       JOIN survey_v1_option
          ON survey_v1_option.OPT_ID =
                survey_v1_user_response_store.OPTION_ID
       JOIN survey_v1_report_index_question_mapper
          ON survey_v1_report_index_question_mapper.QUESTION_ID =
                survey_v1_question.QUESTION_ID
       JOIN survey_v1_report_index
          ON survey_v1_report_index.REPORT_INDEX_ID =
                survey_v1_report_index_question_mapper.REPORT_INDEX_ID
       JOIN survey_v1_defination
          ON survey_v1_defination.SD_ID =
                survey_v1_user_servey_def.SURVEY_DEF_ID
       JOIN manager_survey_user_def_mapping
          ON manager_survey_user_def_mapping.SUSD_ID =
                survey_v1_user_servey_def.USD_ID
       JOIN survey_manager
          ON survey_manager.MANAGER_ID =
                manager_survey_user_def_mapping.MANAGER_ID
 WHERE     survey_v1_option.OPT_VALUE IS NOT NULL
       AND survey_v1_question.QUESTION_TYPE = 'QUESTION_SCALE'
       AND survey_v1_defination.SD_ID = 12
       AND survey_v1_user_servey_def.USD_ID IN (SELECT survey_v1_user_servey_def.USD_ID
                                                  FROM engage_v1.survey_manager AS in_manager
                                                       JOIN
                                                       manager_survey_user_def_mapping
                                                          ON manager_survey_user_def_mapping.MANAGER_ID =
                                                                in_manager.MANAGER_ID
                                                       JOIN
                                                       survey_v1_user_servey_def
                                                          ON survey_v1_user_servey_def.USD_ID =
                                                                manager_survey_user_def_mapping.SUSD_ID
                                                 WHERE in_manager.MANAGER_EMP_ID =
                                                          survey_manager.MANAGER_EMP_ID)
GROUP BY survey_manager.MANAGER_EMP_ID, survey_v1_report_index.INDEX_TYPE;"""
emp_data = pd.read_sql_query(sql=sql_emp,con=database)
emp_data.head()

,MANAGER_EMP_ID,Survey_Type,Avg_Score
0,1,C,4.118326
1,1,I,3.976470
2,10028,C,4.519651
3,10028,I,4.316206
4,10043,C,3.550000


In [24]:
emp_data[emp_data.MANAGER_EMP_ID.isin(['26580'])]

,MANAGER_EMP_ID,Survey_Type,Avg_Score
844,26580,C,4.163106
845,26580,I,4.023252


In [25]:
join_emp_data = pd.merge(join_data,emp_data,left_on=["MANAGER_EMP_ID","INDEX_TYPE_NEW"],right_on=["MANAGER_EMP_ID","Survey_Type"])

In [26]:
join_emp_data.head()

,MANAGER_EMP_ID,VALUE_OLD,INDEX_TYPE_NEW,VALUE_NEW,DIFF,Survey_Type,Avg_Score
0,10028,4.519651,C,4.400000,-0.119651,C,4.519651
1,10028,4.316206,I,4.318182,0.001976,I,4.316206
2,10043,3.550000,C,4.450000,0.900000,C,3.550000
3,10043,3.295455,I,4.136364,0.840909,I,3.295455
4,10060,4.246493,C,4.250000,0.003507,C,4.246493


In [27]:
join_emp_data.drop(['DIFF'],axis=1,inplace=True)

In [28]:
join_emp_data.head()

,MANAGER_EMP_ID,VALUE_OLD,INDEX_TYPE_NEW,VALUE_NEW,Survey_Type,Avg_Score
0,10028,4.519651,C,4.400000,C,4.519651
1,10028,4.316206,I,4.318182,I,4.316206
2,10043,3.550000,C,4.450000,C,3.550000
3,10043,3.295455,I,4.136364,I,3.295455
4,10060,4.246493,C,4.250000,C,4.246493


In [29]:
join_emp_data["OLD_DASH"]=(join_emp_data.VALUE_OLD+join_emp_data.Avg_Score)/2
join_emp_data["NEW_DASH"]=(join_emp_data.VALUE_NEW+join_emp_data.Avg_Score)/2
join_emp_data.head()

,MANAGER_EMP_ID,VALUE_OLD,INDEX_TYPE_NEW,VALUE_NEW,Survey_Type,Avg_Score,OLD_DASH,NEW_DASH
0,10028,4.519651,C,4.400000,C,4.519651,4.519651,4.459825
1,10028,4.316206,I,4.318182,I,4.316206,4.316206,4.317194
2,10043,3.550000,C,4.450000,C,3.550000,3.550000,4.000000
3,10043,3.295455,I,4.136364,I,3.295455,3.295455,3.715909
4,10060,4.246493,C,4.250000,C,4.246493,4.246493,4.248246


In [30]:
join_emp_data["DIFF"]=join_emp_data.NEW_DASH-join_emp_data.OLD_DASH

In [31]:
join_emp_data.drop(['INDEX_TYPE_NEW'],axis=1,inplace=True)
join_emp_data.head()

,MANAGER_EMP_ID,VALUE_OLD,VALUE_NEW,Survey_Type,Avg_Score,OLD_DASH,NEW_DASH,DIFF
0,10028,4.519651,4.400000,C,4.519651,4.519651,4.459825,-0.059825
1,10028,4.316206,4.318182,I,4.316206,4.316206,4.317194,0.000988
2,10043,3.550000,4.450000,C,3.550000,3.550000,4.000000,0.450000
3,10043,3.295455,4.136364,I,3.295455,3.295455,3.715909,0.420455
4,10060,4.246493,4.250000,C,4.246493,4.246493,4.248246,0.001754


In [32]:
#join_emp_data.to_excel("F:\GMR_MANAGER_COMPARE_REPORT.xlsx",sheet_name="comparision")

In [33]:
join_emp_data.sort_values("DIFF").head()

,MANAGER_EMP_ID,VALUE_OLD,VALUE_NEW,Survey_Type,Avg_Score,OLD_DASH,NEW_DASH,DIFF
670,2967,4.267409,0.000000,C,4.267409,4.267409,2.133705,-2.133705
671,2967,4.197468,0.000000,I,4.197468,4.197468,2.098734,-2.098734
658,2772,3.802752,0.000000,I,3.802752,3.802752,1.901376,-1.901376
657,2772,3.782828,0.000000,C,3.782828,3.782828,1.891414,-1.891414
1015,7209,4.500000,1.611111,C,4.500000,4.500000,3.055556,-1.444444


In [34]:
join_emp_data.DIFF.describe()

count    1140.000000
mean        0.043238
std         0.315114
min        -2.133705
25%        -0.125000
50%         0.043290
75%         0.247328
max         1.059091
Name: DIFF, dtype: float64

In [35]:
join_emp_data.MANAGER_EMP_ID.dtype

dtype('O')

In [36]:
join_emp_data[join_emp_data.MANAGER_EMP_ID.isin(['2772','20530','2967','26580','1036'])]

,MANAGER_EMP_ID,VALUE_OLD,VALUE_NEW,Survey_Type,Avg_Score,OLD_DASH,NEW_DASH,DIFF
28,1036,4.184071,4.600000,C,4.184071,4.184071,4.392035,0.207965
29,1036,4.040078,4.227273,I,4.040078,4.040078,4.133675,0.093597
399,20530,3.543933,4.800000,C,3.543933,3.543933,4.171967,0.628033
400,20530,3.604563,4.636364,I,3.604563,3.604563,4.120463,0.515900
657,2772,3.782828,0.000000,C,3.782828,3.782828,1.891414,-1.891414
658,2772,3.802752,0.000000,I,3.802752,3.802752,1.901376,-1.901376
670,2967,4.267409,0.000000,C,4.267409,4.267409,2.133705,-2.133705
671,2967,4.197468,0.000000,I,4.197468,4.197468,2.098734,-2.098734


In [37]:
old_req = pd.read_excel("C:\Users\BNSPL-2\Documents\Received Files\manager dashboard.xlsx",sheetname="Sheet1")

In [38]:
old_req = old_req[old_req["Unnamed: 4"]=='n'].MANAGER_EMP_ID.values

In [39]:
old_req=old_req.astype(str)

In [47]:
old_req.shape

(31,)

In [50]:
adjust_old_req = [item for item in old_req if item not in ["17733","19809","2212","2862","7746"]]
len(adjust_old_req)

26

In [63]:
new_gmr_data=join_emp_data[join_emp_data.MANAGER_EMP_ID.isin(adjust_old_req)]
old_gmr_data1=join_emp_data[~join_emp_data.MANAGER_EMP_ID.isin(adjust_old_req)]
old_gmr_data=emp_data[~emp_data.MANAGER_EMP_ID.isin(adjust_old_req)]
print(new_gmr_data.shape,old_gmr_data.shape,old_gmr_data1.shape)

((52, 8), (1405, 3), (1088, 8))


In [64]:
old_gmr_data.head(2)

,MANAGER_EMP_ID,Survey_Type,Avg_Score
0,1,C,4.118326
1,1,I,3.976470


In [65]:
new_gmr_data.head(2)

,MANAGER_EMP_ID,VALUE_OLD,VALUE_NEW,Survey_Type,Avg_Score,OLD_DASH,NEW_DASH,DIFF
8,10067,3.719818,4.250000,C,3.719818,3.719818,3.984909,0.265091
9,10067,3.747863,4.136364,I,3.747863,3.747863,3.942113,0.194250


In [66]:
#old_gmr_data.drop(["VALUE_OLD","VALUE_NEW","Avg_Score","DIFF"],axis=1,inplace=True)
new_gmr_data.drop(["VALUE_OLD","VALUE_NEW","Avg_Score","DIFF"],axis=1,inplace=True)

c:\python27\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [67]:
old_gmr_data[old_gmr_data.MANAGER_EMP_ID=='26580']

,MANAGER_EMP_ID,Survey_Type,Avg_Score
844,26580,C,4.163106
845,26580,I,4.023252


In [68]:
print(old_gmr_data.Avg_Score.dtype,new_gmr_data.OLD_DASH.dtype)
print(old_gmr_data.Avg_Score.shape,new_gmr_data.OLD_DASH.shape)

(dtype('float64'), dtype('float64'))
((1405,), (52,))


In [69]:
jsonMap = {}

for row in new_gmr_data.itertuples():
        if(jsonMap.get(row[1])!=None):
            data = jsonMap.get(row[1])
            data.update({row[2]:round(row[3],2)})
        else:
            jsonMap.update({row[1]:{row[2]:round(row[4],2)}})


print(len(jsonMap.keys()))

26


In [70]:
for row in old_gmr_data.itertuples():
        if(jsonMap.get(row[1])!=None):
            data = jsonMap.get(row[1])
            data.update({row[2]:round(row[3],2)})
        else:
            jsonMap.update({row[1]:{row[2]:round(row[3],2)}})

            
print(len(jsonMap.keys()))

729


In [71]:
json.dumps(jsonMap)

'{"3922": {"I": 4.27, "C": 4.32}, "3926": {"I": 4.13, "C": 4.19}, "3927": {"I": 4.2, "C": 4.39}, "18558": {"I": 4.0, "C": 4.23}, "18559": {"I": 4.31, "C": 4.31}, "23367": {"I": 3.91, "C": 4.14}, "10105": {"I": 4.04, "C": 4.17}, "8090": {"I": 4.39, "C": 4.62}, "8884": {"I": 3.89, "C": 4.0}, "10102": {"I": 4.0, "C": 4.0}, "3552": {"I": 4.3, "C": 4.54}, "5826": {"I": 3.64, "C": 3.8}, "3256": {"I": 3.77, "C": 4.01}, "1945": {"I": 4.19, "C": 4.25}, "24710": {"I": 3.56, "C": 3.65}, "24717": {"I": 3.5, "C": 3.9}, "19408": {"I": 2.77, "C": 3.05}, "19155": {"I": 3.9, "C": 4.13}, "22184": {"I": 3.83, "C": 4.08}, "2858": {"I": 3.8, "C": 3.87}, "14352": {"I": 4.0, "C": 4.0}, "19376": {"I": 3.98, "C": 4.15}, "1793": {"I": 4.0, "C": 4.05}, "1790": {"I": 3.72, "C": 3.9}, "6758": {"I": 3.09, "C": 3.35}, "9477": {"I": 3.91, "C": 3.9}, "24308": {"I": 4.0, "C": 3.95}, "15759": {"I": 3.9, "C": 4.1}, "11538": {"I": 3.55, "C": 3.65}, "22163": {"I": 3.82, "C": 4.2}, "22160": {"I": 3.6, "C": 3.75}, "5468": {"

In [75]:
f = open('F:\GMR Manager New Report\manager_json.json', 'w')
json.dump(jsonMap,f)

In [76]:
jsonMap['7746']

{'C': 4.2, 'I': 3.99}